<h2 style="color:blue" align="center">Plain Deep Neural Network (DNN)</h2>

#### Import the necessary libraries

In [98]:
import tensorflow as tf 
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import time, math 

from tensorflow import keras 
from keras.models import Sequential
from keras.layers import Dense 
from keras.callbacks import Callback, CSVLogger 

#### Load and split the dataset into training and testing set

MNIST - Handwritten digits recognition

In [99]:
#(X_train,y_train),(X_test,y_test) = keras.datasets.mnist.load_data()

<img src="Figures/MNIST-Handwritten digits.png" height=450 width=450/>

MNIST - Fashion dataset

In [100]:
(X_train,y_train),(X_test,y_test) = keras.datasets.fashion_mnist.load_data()

<img src="Figures/MNIST-fashion.png" height=400 width=400/>

#### Flattening

In [101]:
# For MNIST handwritten digits and Fashion dataset
X_train_flattened = X_train.reshape(len(X_train), 28 * 28)
X_test_flattened = X_test.reshape(len(X_test),  28 * 28) 

# For CIFAR-10 and CIFAR-100
#X_train_flattened = X_train.reshape(len(X_train), 32 * 32)
#X_test_flattened = X_test.reshape(len(X_test),  32 * 32) 

#### Normalize the input dataset

In [102]:
X_train_normalized = X_train_flattened / 255
X_test_normalized = X_test_flattened / 255

#### Define DNN model with two layers, optimizer, metrics, and loss function

In [103]:
np.random.seed(3)

def get_model(): 
    model = Sequential([
        # input layer 784 neurons to first hidden layer with 64 neurons
        Dense(64, input_shape = (784,), activation='relu'), 
        # first hidden layer to second hidden layer
        Dense(64, activation='relu'),  
        # Output layer with 10 neurons
        Dense(10, activation='softmax')
    ])
    
    # General SGD
    #opti = keras.optimizers.SGD(learning_rate=0.01)
    
    # SGD with momentum
    #opti = keras.optimizers.SGD(learning_rate=0.01, momentum=0.6)
    
    # SGD with Nesterov momentum 
    #opti = keras.optimizers.SGD(learning_rate=0.01, momentum=0.6, nesterov=True)
    
    # RMSprop 
    #opti = keras.optimizers.RMSprop(learning_rate=0.001, momentum=0.6)
    
    # Adam
    opti = keras.optimizers.Adam(learning_rate=0.001) 
    
    # Adamax
    #opti = keras.optimizers.Adamax(learning_rate=0.001) 
    
    model.compile(
        optimizer = opti,
        loss = 'sparse_categorical_crossentropy',
        metrics = ['accuracy']
    )
    
    return model

#### Custom callbacks

For generic optimizer model

In [104]:
class CustomCallbackGeneric(Callback):  
    # Training stop criteria
    stop_at = 0.99
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy')> self.stop_at):  
            self.model.stop_training = True

For HM based optimizer model

In [105]:
class CustomCallbackHM(Callback):  
    # Stop the algorithm when the following accuracy reached 
    stop_at = 0.99
    
    # to update weights for batch gradient desecent
    batch_size = 60000    
    iteration = 0    
    training_set_size = 0
    update_every_fold = 0  
    
    initial_weights = 0
    previous_weights = 0
    call_hm = 0 
     
    def on_train_begin(self, logs=None):
        self.initial_weights = model_hm.get_weights() 
        self.initial_weights = np.array(self.initial_weights,dtype=object)
        self.previous_weights = self.initial_weights
        # Harmonic mean based weights calculation
        self.call_hm = np.vectorize(self.apply_hm) 
        #Determining number of updates for mini-batch gradient every epoch
        self.training_set_size = X_train_normalized.shape[0]
        self.update_every_fold = self.fold_calc(self.batch_size, self.training_set_size)
             
    def on_epoch_begin(self, epoch,  logs=None): 
        self.iteration = 1   
        
    def on_train_batch_end(self, batch, logs=None): 
        if self.iteration%self.update_every_fold == 0:  
            counter = 0
            num_layers = len(model_hm.layers)  
            current_weights = model_hm.get_weights()
            current_weights = np.array(current_weights,dtype=object)        

            for i in range(num_layers):  
                # Harmonic mean based weights calculation
                current_weights[counter] = self.call_hm(self.previous_weights[counter], current_weights[counter])
                counter = counter + 2
            
            # Updating the model with new weights
            updated = current_weights.tolist()   
            model_hm.set_weights(updated)
            self.previous_weights = current_weights
        self.iteration = self.iteration + 1  
    
    def on_epoch_end(self, epoch, logs={}):
        # Training stop criteria
        if(logs.get('accuracy')> self.stop_at):  
            self.model.stop_training = True

    def apply_hm(self, v1, v2):     
        if v1==0 or v2==0:
            return v2
        elif v1>0 and v2>0:
            hm = 2*v1*v2/(v1+v2)
            min1 = min(v1,v2)
            diff = abs(hm-min1)
            if v2 > v1:
                return v2 + diff
            else:
                return v2 - diff
        elif v1<0 and v2<0:
            hm = 2*v1*v2/(v1+v2)
            max1 = max(v1,v2)
            diff = abs(hm-max1)
            if v2 > v1:
                return v2 + diff
            else:
                return v2 - diff
        else:
            return v2  
        
    def fold_calc(self, batch_size, training_set_size): 
        total_fold = int(math.ceil(math.log2(training_set_size)))
        #print("total_fold   :", total_fold) 
        num_batch = math.ceil(training_set_size / batch_size)
        #print("num_batch   :", num_batch)
        batch_fold = int(math.floor(math.log2(batch_size)))
        #print("batch_fold   :", batch_fold)
        update_every_fold = (total_fold-batch_fold)    
        #print("update_every_fold   :", update_every_fold)
        num_updates = num_batch/update_every_fold
        #print("num_updates   :", num_updates)
        return update_every_fold

To record loss and accuracy in CSV file

In [106]:
logger_generic_model = CSVLogger('Generic_model_MNIST.csv', append=False, separator=',')
logger_hm_model = CSVLogger('HM_model_MNIST.csv', append=False, separator=',')

#### Training

Generic opimizer model

In [107]:
model_wihtout_hm = get_model() 
st = time.time()
model_wihtout_hm.fit(X_train_normalized, y_train, epochs = 100, verbose=1, callbacks=[CustomCallbackGeneric(), logger_generic_model], batch_size=60000) 
et = time.time()
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')

Epoch 1/100
1/1 [==============================] - 1s 663ms/step - loss: 2.3085 - accuracy: 0.1054
Epoch 2/100
1/1 [==============================] - 0s 168ms/step - loss: 2.1792 - accuracy: 0.2074
Epoch 3/100
1/1 [==============================] - 0s 220ms/step - loss: 2.0838 - accuracy: 0.3064
Epoch 4/100
1/1 [==============================] - 0s 171ms/step - loss: 1.9965 - accuracy: 0.3773
Epoch 5/100
1/1 [==============================] - 0s 172ms/step - loss: 1.9103 - accuracy: 0.4345
Epoch 6/100
1/1 [==============================] - 0s 161ms/step - loss: 1.8247 - accuracy: 0.4777
Epoch 7/100
1/1 [==============================] - 0s 190ms/step - loss: 1.7399 - accuracy: 0.5163
Epoch 8/100
1/1 [==============================] - 0s 163ms/step - loss: 1.6573 - accuracy: 0.5542
Epoch 9/100
1/1 [==============================] - 0s 211ms/step - loss: 1.5775 - accuracy: 0.5867
Epoch 10/100
1/1 [==============================] - 0s 228ms/step - loss: 1.5008 - accuracy: 0.6072
Epoch 11/

1/1 [==============================] - 0s 186ms/step - loss: 0.4553 - accuracy: 0.8424
Epoch 84/100
1/1 [==============================] - 0s 205ms/step - loss: 0.4517 - accuracy: 0.8439
Epoch 85/100
1/1 [==============================] - 0s 161ms/step - loss: 0.4510 - accuracy: 0.8440
Epoch 86/100
1/1 [==============================] - 0s 156ms/step - loss: 0.4502 - accuracy: 0.8439
Epoch 87/100
1/1 [==============================] - 0s 172ms/step - loss: 0.4469 - accuracy: 0.8456
Epoch 88/100
1/1 [==============================] - 0s 154ms/step - loss: 0.4466 - accuracy: 0.8452
Epoch 89/100
1/1 [==============================] - 0s 147ms/step - loss: 0.4454 - accuracy: 0.8457
Epoch 90/100
1/1 [==============================] - 0s 172ms/step - loss: 0.4424 - accuracy: 0.8467
Epoch 91/100
1/1 [==============================] - 0s 172ms/step - loss: 0.4425 - accuracy: 0.8465
Epoch 92/100
1/1 [==============================] - 0s 156ms/step - loss: 0.4407 - accuracy: 0.8472
Epoch 93/100


HM based optimizer model

In [108]:
model_hm = get_model() 
st = time.time()
model_hm.fit(X_train_normalized, y_train, epochs = 100, verbose=1, callbacks=[CustomCallbackHM(),logger_hm_model], batch_size=60000) 
et = time.time()
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')

Epoch 1/100
1/1 [==============================] - 1s 718ms/step - loss: 2.3422 - accuracy: 0.1530
Epoch 2/100
1/1 [==============================] - 0s 219ms/step - loss: 2.1698 - accuracy: 0.2272
Epoch 3/100
1/1 [==============================] - 0s 219ms/step - loss: 2.0489 - accuracy: 0.3683
Epoch 4/100
1/1 [==============================] - 0s 203ms/step - loss: 1.9210 - accuracy: 0.4486
Epoch 5/100
1/1 [==============================] - 0s 201ms/step - loss: 1.7834 - accuracy: 0.5155
Epoch 6/100
1/1 [==============================] - 0s 232ms/step - loss: 1.6441 - accuracy: 0.5777
Epoch 7/100
1/1 [==============================] - 0s 202ms/step - loss: 1.5142 - accuracy: 0.6190
Epoch 8/100
1/1 [==============================] - 0s 204ms/step - loss: 1.3960 - accuracy: 0.6402
Epoch 9/100
1/1 [==============================] - 0s 236ms/step - loss: 1.2885 - accuracy: 0.6523
Epoch 10/100
1/1 [==============================] - 0s 229ms/step - loss: 1.1930 - accuracy: 0.6600
Epoch 11/

1/1 [==============================] - 0s 208ms/step - loss: 0.4282 - accuracy: 0.8515
Epoch 84/100
1/1 [==============================] - 0s 199ms/step - loss: 0.4266 - accuracy: 0.8521
Epoch 85/100
1/1 [==============================] - 0s 203ms/step - loss: 0.4250 - accuracy: 0.8528
Epoch 86/100
1/1 [==============================] - 0s 213ms/step - loss: 0.4234 - accuracy: 0.8531
Epoch 87/100
1/1 [==============================] - 0s 205ms/step - loss: 0.4218 - accuracy: 0.8538
Epoch 88/100
1/1 [==============================] - 0s 199ms/step - loss: 0.4201 - accuracy: 0.8539
Epoch 89/100
1/1 [==============================] - 0s 194ms/step - loss: 0.4186 - accuracy: 0.8545
Epoch 90/100
1/1 [==============================] - 0s 206ms/step - loss: 0.4173 - accuracy: 0.8546
Epoch 91/100
1/1 [==============================] - 0s 190ms/step - loss: 0.4166 - accuracy: 0.8545
Epoch 92/100
1/1 [==============================] - 0s 196ms/step - loss: 0.4173 - accuracy: 0.8541
Epoch 93/100


Model summary

In [109]:
model_hm.summary()

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_51 (Dense)            (None, 64)                50240     
                                                                 
 dense_52 (Dense)            (None, 64)                4160      
                                                                 
 dense_53 (Dense)            (None, 10)                650       
                                                                 
Total params: 55,050
Trainable params: 55,050
Non-trainable params: 0
_________________________________________________________________


###### Testing the model

Generic opimizer model

In [110]:
model_wihtout_hm.evaluate(X_test_normalized, y_test)

313/313 [==============================] - 0s 994us/step - loss: 0.4637 - accuracy: 0.8381


[0.46369126439094543, 0.838100016117096]

HM based optimizer model

In [111]:
model_hm.evaluate(X_test_normalized, y_test)

313/313 [==============================] - 0s 987us/step - loss: 0.4384 - accuracy: 0.8482


[0.4384474754333496, 0.8482000231742859]